In [ ]:
from __future__ import print_function

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
import os 
import sys
import gzip
import pickle
from urllib import urlretrieve

def maybe_download(filename, expected_bytes, force=False):
    """Download a file if not present, and make sure it's the right size."""
    dest_filename = os.path.join(data_root, filename)
    if force or not os.path.exists(dest_filename):
        print('Attempting to download:', filename) 
        filename, _ = urlretrieve(url + filename, dest_filename)
        print('\nDownload Complete!')
    
    statinfo = os.stat(dest_filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', dest_filename)
    else:
        raise Exception(
            'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
    return dest_filename

data_root = 'data/'
url = 'http://deeplearning.net/data/mnist/'
train_filename = maybe_download('mnist.pkl.gz', 16168813)

In [ ]:
with gzip.open('data/mnist.pkl.gz', 'rb') as f:
    (X_train, y_train), (X_val, y_val), (X_test, y_test) =  pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
# plot 4 images as gray scale
plt.subplot(221)
plt.imshow(X_train[0].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(X_train[1].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(X_train[2].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(X_train[3].reshape(28,28), cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

In [ ]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [ ]:
batch_size = 128

def batch_generator(X, y, y_labels, batch_size):
    l = X.shape[0]
    def another():
        indexes = np.random.randint(low=0, high=l, size=batch_size)
        return [
            X.take(indexes, axis=0),
            y[indexes],
            y_labels[indexes]
        ]
            
    while True:
        yield another()

In [ ]:
import matplotlib.pyplot as plt
def plot_images(X):# plot 4 images as gray scale
    plt.subplot(221)
    plt.imshow(X[0].reshape(28,28), cmap=plt.get_cmap('gray'))
    plt.subplot(222)
    plt.imshow(X[1].reshape(28,28), cmap=plt.get_cmap('gray'))
    plt.subplot(223)
    plt.imshow(X[2].reshape(28,28), cmap=plt.get_cmap('gray'))
    plt.subplot(224)
    plt.imshow(X[3].reshape(28,28), cmap=plt.get_cmap('gray'))
    # show the plot
    plt.show()

In [ ]:
def new_batch(X, y, ids):
    n,d = X.shape
    indexes = np.array([False] * n)
    for i in ids:
        indexes = np.logical_or(indexes, y == i)
    X = X[indexes,]
    y_labels = y[indexes]
    y = y[indexes]
    X, y = reformat(X, y)
    return([X, y, y_labels])

In [ ]:
numbers = range(10)
batches = []
data = []

"""
for num in numbers:
    X, y = X_train[y_train == num,], y_train[y_train == num] 
    X_neg, y_neg = X_train[y_train != num,][:10000,], y_train[y_train != num][:10000,]
    y_labels = y_train[y_train == num]  
    y_labels_neg = y_train[y_train != num][:10000,]  
    X = np.concatenate((X, X_neg), axis=0)
    y = np.concatenate((y, y_neg), axis=0)
    y_labels = np.concatenate((y_labels, y_labels_neg), axis=0)
    X, y = reformat(X, y)


    data.append([X, y, y_labels])
    batches.append(batch_generator(X, y, y_labels, batch_size=batch_size))
    
"""

batches = []
data = []

def generate_batches(proto):
    for ids in proto:
        [X, y, y_labels] = new_batch(X_train, y_train, ids)
        data.append([X, y, y_labels])
        batches.append(batch_generator(X, y, y_labels, batch_size=batch_size))
    

In [ ]:
generate_batches([[0],[1],[2],[3],[4],[5],[6],[7],[8],[9]]) 

In [ ]:
[X, y, y_labels] = new_batch(X_train, y_train, range(10))
train_batches = batch_generator(X, y, y_labels, batch_size=batch_size)

In [ ]:
X, y, y_labels = batches[0].next()

In [ ]:
plot_images(X)

# Model

In [ ]:
batch_size = 128

In [ ]:
class Model:

    def __init__(self, l1 = 128, l2 = 128, l1_loss = 0.0, l2_loss = 0.0, mode = 'normal', pretrained=False, ewc = 20e6):
        
        # Network Parameters
        n_hidden_1 = l1 # 1st layer number of features
        n_hidden_2 = l2 # 2nd layer number of features
        n_input = 784 # MNIST data input (img shape: 28*28)
        n_classes = 10 # MNIST total classes (0-9 digits)
        
        # tf Graph input
        self.tf_x = tf.placeholder("float", [None, n_input])
        self.tf_y = tf.placeholder("float", [None, n_classes])
        self.tf_labels = tf.placeholder("int32", [None])
        self.active_class = tf.placeholder("int32")
        
        self.ewc = tf.constant(ewc)
        self.l1_loss = tf.constant(l1_loss)
        self.l2_loss = tf.constant(l2_loss)
        
        # Create model
        def multilayer_perceptron(x, weights):
            # Hidden layer with RELU activation
            layer_1 = tf.matmul(x, weights['h1'])
            layer_1 = tf.nn.relu(layer_1)
            # Hidden layer with RELU activation
            layer_2 = tf.matmul(layer_1, weights['h2'])
            layer_2 = tf.nn.relu(layer_2)
            # Output layer with linear activation
            out_layer = tf.matmul(layer_2, weights['out'])
            return out_layer
        
        self.multilayer_perceptron = multilayer_perceptron
        
        # Store layers weight
        self.weights = {
            'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
            'h1e': tf.Variable(tf.random_normal([n_input, n_hidden_1]), trainable=False),
            'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
            'h2e': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), trainable=False)#,
            #'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes])),
            #'oute': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), trainable=False)
        }
        
        if pretrained == False:
            self.weights['out'] = tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name="vout")
        if pretrained == True:
            self.weights['out'] = tf.Variable(tf.random_normal([n_hidden_2, n_classes]), trainable=False, name="vout")

        self.saver = tf.train.Saver({"vout": self.weights['out'] })
                 
        self.fisher = {
            'h1': tf.Variable(tf.zeros([n_input, n_hidden_1])),
            'h2': tf.Variable(tf.zeros([n_hidden_1, n_hidden_2]))#,
            #'out': tf.Variable(tf.zeros([n_hidden_2, n_classes]))
        }

        # Construct model
        self.pred = self.multilayer_perceptron(self.tf_x, self.weights)

        # Define loss and optimizer
        if mode == 'normal':
            cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.pred, labels=self.tf_y)) 
        if mode == 'sparse':
            cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.pred, labels=self.tf_labels))  
        if mode == 'continuous':
            cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=self.pred[:, 0:self.active_class+1],
                labels=self.tf_y[:, 0:self.active_class+1]))  
            
        cost = cost + self.l1_loss*tf.reduce_sum(tf.abs(self.weights['h1']))
        cost = cost + self.l1_loss*tf.reduce_sum(tf.abs(self.weights['h2']))
        cost = cost + self.l1_loss*tf.reduce_sum(tf.abs(self.weights['h1']))
        cost = cost + self.l2_loss*tf.nn.l2_loss(self.weights['h1'])
        cost = cost + self.l2_loss*tf.nn.l2_loss(self.weights['h2'])
        cost = cost + self.l2_loss*tf.nn.l2_loss(self.weights['out'])
        self.cost = cost
        
        self.optimizer = tf.train.AdamOptimizer(0.01)
        self.minimize_cost = self.optimizer.minimize(self.cost) 
        
        self.grads = self.optimizer.compute_gradients(self.cost, [self.weights['h1'], self.weights['h2'], self.weights['out']]) 

        ewc_cost = self.ewc * tf.nn.l2_loss(tf.multiply(self.fisher['h1'], tf.subtract(self.weights['h1'], self.weights['h1e'])))
        ewc_cost = ewc_cost + self.ewc * tf.nn.l2_loss(tf.multiply(self.fisher['h2'], tf.subtract(self.weights['h2'], self.weights['h2e'])))
        #ewc_cost = ewc_cost + self.ewc * tf.nn.l2_loss(tf.multiply(self.fisher['out'], tf.subtract(self.weights['out'], self.weights['oute'])))
        self.ewc_cost = ewc_cost
        
        self.objective = self.cost + self.ewc_cost
        self.minimize_cost_ewc = self.optimizer.minimize(self.objective)
        
        # Initializing the variables
        self.init = tf.global_variables_initializer()        
     

### Helper functions for training

In [ ]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
                    / predictions.shape[0])

In [ ]:
def restore(sess):
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoint_directory/{}'.format(model_name)))
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time

class Progress(object):
    
    def __init__(self, max):
        self.p = IntProgress(max = max)
        self.p.description = 'Running'
        self.stime = time.time()
        self.last_delta = 0
        self.max = max
        display(self.p)
    
    def update(self, step):
        self.p.value = step
        delta = int(time.time() - self.stime)
        if delta % 5 == 0 and delta > self.last_delta:
            self.last_delta = delta
            time_remaining = int(delta * (self.max - step) / step)
            self.p.description = '{0}:{1}'.format(time_remaining / 60, time_remaining % 60 )                   

In [ ]:
max_steps = 3001
period_print = 500
period_stat = 100

def train(model, batches_list, data = None, EWC = False, stat = False, max_steps = 3001):
    
    acc_track = []
    #for i in range(len(data)+1):
    acc_track.append([])
        
    p = Progress(max_steps*len(batches_list))
    pr = 0
    
    for ind, batches in enumerate(batches_list):
        print("Training on batches{0}\n".format(ind+1))
        try:
            for step in range(max_steps):
                
                pr = pr+1
                p.update(pr)
                
                X, y, y_labels = batches.next()
                train_d = {model.tf_x : X, model.tf_y : y, model.tf_labels : y_labels, model.active_class : ind}

                if EWC ==  False:
                    _ = sess.run(model.minimize_cost, feed_dict=train_d)
                if EWC ==  True:
                    _ = sess.run(model.minimize_cost_ewc, feed_dict=train_d)

                if step % period_stat == 0:
                    #for i, d in enumerate(data):
                    #    acc_track[i].append(test_acc(model, d[0], d[1]))
                    
                    X_t, y_t, y_t_labels = train_batches.next()
                    acc_track[0].append(test_acc(model, X_t, y_t))

                if (step % period_print == 0) and (stat == True):
                    test_d = {model.tf_x : X, model.tf_y : y}
                    l,ll, train_pred = sess.run([model.cost, model.ewc_cost, model.pred], feed_dict=test_d)
                    print(
                        "Step {0}: \n loss: {1} {2} \n accuracy: {3}% \n".format(
                            step,
                            l,
                            ll,
                            test_acc(model, X_t, y_t)
                        )
                    )
            set_constraint(model, X, y, y_labels, ind)

        except KeyboardInterrupt:
            set_constraint(model, X, y, y_labels, ind)
            print('training interrupted')

    return acc_track

In [ ]:
def test_acc(model, X, y):
    #X, y = batches.next()
    test_d = {model.tf_x : X, model.tf_y : y}
    prediction = sess.run([model.pred], feed_dict=test_d)
    return accuracy(prediction[0], y)  

In [ ]:
def get_gradient(model, batches):
    X, y = batches.next()
    test_d = {model.tf_x : X, model.tf_y : y}
    gradient = sess.run([model.grads], feed_dict=test_d)
    return gradient

In [ ]:
# Привязать к данным
def set_constraint(model, X, y, y_labels, id):
    # Вычисляем Градиент по весам
    test_d = {model.tf_x : X, model.tf_y : y, model.tf_labels : y_labels, model.active_class : id}
    g = sess.run([model.grads], feed_dict=test_d)
    #return gradient
    g1, w1 = g[0][0]
    g2, w2 = g[0][1]
    g3, w3 = g[0][2]
    # Забираем старые матрицы фишера
    # Возводим в квадрат
    with sess.as_default():
        f1 = 0.3 * np.square(g1) + 0.7 * model.fisher['h1'].eval()
        f2 = 0.3 * np.square(g2) + 0.7 * model.fisher['h2'].eval()
        #f3 = 0.3 * np.square(g3) + 0.7 * model.fisher['out'].eval()

    #print(np.max(f1),np.max(f2),np.max(f3))
    with sess.as_default():
        # Обновляем коэффициенты матрицы фишера в графе
        model.fisher['h1'].assign(f1).eval()
        model.fisher['h2'].assign(f2).eval()
        #model.fisher['out'].assign(f3).eval()
        # Обновляем центральные веса в графе
        model.weights['h1e'].assign(w1).eval()
        model.weights['h2e'].assign(w2).eval()
        #model.weights['oute'].assign(w3).eval()

In [ ]:
def plot(acc_track, title):
    %matplotlib inline
    import matplotlib.pyplot as plt
    plt.rc("font", size=10)
    fig = plt.figure()
    fig.suptitle(title, fontsize=14)
    axes = plt.gca()
    axes.set_ylim([0,100])
    l = len(acc_track[0])
    for ind, acc in enumerate(acc_track):
        plt.plot(range(l), acc_track[ind], label='Number {0}'.format(ind))
    
    plt.ylabel('accuracy, %')
    plt.xlabel('number of training batches x100')
    plt.legend(loc=4, borderaxespad=0.)
    plt.show()

In [ ]:
model = Model(256, 256, l1_loss = 0.0, l2_loss = 0.0001, mode = 'normal', pretrained = False)
#tf.reset_default_graph()
sess = tf.Session()
sess.run(model.init)

stats = []

In [ ]:
stats.append(train(model, [train_batches], data, EWC=False, stat=True, max_steps = 3001))

In [ ]:
save_path = model.saver.save(sess, "/tmp/model.ckpt")

In [ ]:
plot(stats[0], title = "Fig.1. (l1 = 256, l2 = 256) initial pretraining")

In [ ]:
save_path

In [2]:
model = Model(256, 256, l1_loss = 0.0, l2_loss = 0.0, mode = 'normal', pretrained = True, ewc = 1e15)
#tf.reset_default_graph()
sess = tf.Session()
sess.run(model.init)

stats = []

NameError: name 'Model' is not defined

In [ ]:
model.saver.restore(sess, save_path)

In [ ]:
stats.append(train(model, b, data, EWC=True, stat=True, max_steps = 501))

In [ ]:
b = []

for i in range(10):
    for j in range(2*i):
        b.append(batches[np.random.randint(low = 0, high = i+1, size=1, dtype='l')[0]
])
        
len(b)

In [ ]:
stats.append(train(model, b, data, EWC=False, stat=True))

In [ ]:
plot(stats[0], title = "Fig.3. (l1 = 64, l2 = 256) with EWC ")

In [ ]:
b = []

for i in range(10):
    for j in range(2*i):
        b.append(batches[np.random.randint(low = 0, high = i+1, size=1, dtype='l')[0]
])
        
len(b)

In [ ]:
model = Model(1024, 128, l1_loss = 0.0, l2_loss = 0.0001, mode = 'normal')
sess.run(model.init)

In [ ]:
stats.append(train(model, b, data, EWC=True, stat=True))

In [ ]:
plot(stats[1], title = "Fig.3. (l1 = 64, l2 = 256) with EWC ")